In [1]:
from ipumspy import readers, ddi, IpumsApiClient, AggregateDataExtract, Dataset, DatasetMetadata
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
from matplotlib.ticker import FuncFormatter
import arcgis
from arcgis.gis import GIS
from arcgis.map import Map
from arcgis.raster import Raster
from arcgis.features import GeoAccessor
import geopandas as gpd
import glob
from zipfile import ZipFile
import earthaccess as ea
import requests
import pprint

In [9]:
arcgis.__version__

'2.4.0'

In [2]:
#Load the IPUMS API key from the .env file

load_dotenv()

IPUMS_API_KEY = os.getenv("IPUMS_API_KEY")
ipums = IpumsApiClient(IPUMS_API_KEY)

In [ ]:
#Submit pulling to IPUMS 

extract = AggregateDataExtract(
    collection="nhgis",
    description="Puerto Rico 2010-2020 vacancy",
    datasets=[
        Dataset(name="2010_SF1a", data_tables=["H1", "H3"], geog_levels=["block"]),
        Dataset(name = "2020_DHCa", data_tables = ["H1", "H3"], geog_levels = ["block"])
    ],
    geographic_extents=["720"],
    shapefiles=["720_block_2020_tl2020"]
)

In [ ]:
#Getting shapefile metadata

ds = ipums.get_metadata(DatasetMetadata("nhgis", "2010_SF1a"))
for page in ipums.get_metadata_catalog("nhgis", metadata_type="shapefiles"):
    for shapefile in page["data"]:
        if shapefile["extent"] == "Puerto Rico" and shapefile["geographicLevel"] == "Block":
            print( "Name: " + shapefile["name"] + " | Year: " + shapefile["year"])

Name: 720_block_2000_tl2010 | Year: 2000
Name: 720_block_2010_tl2010 | Year: 2010
Name: 720_block_2010_tl2020 | Year: 2010
Name: 720_block_2020_tl2020 | Year: 2020


In [ ]:
#Submit the extract request
ipums.submit_extract(extract)
print(f"Extract ID: {extract.extract_id}")

#Wait for the extract to finish
ipums.wait_for_extract(extract)

#Download the extract
current = os.getcwd()
DOWNLOAD_DIR = os.path.join(f"{current}/data")
ipums.download_extract(extract, download_dir=DOWNLOAD_DIR)

Extract ID: 13


In [3]:
#Reading from zip file

current = os.getcwd()
DOWNLOAD_DIR = os.path.join(f"{current}/data")

file_list = os.listdir(DOWNLOAD_DIR)
csv_zip = [f for f in file_list if f.endswith('_csv.zip')]
shape_zip = [f for f in file_list if f.endswith('_shape.zip')]
csv = f"{DOWNLOAD_DIR}/{csv_zip[0]}" 
shapefile = f"{DOWNLOAD_DIR}/{shape_zip[0]}"
csv_data = ZipFile(csv).namelist()
shape_data = ZipFile(shapefile).namelist()

In [ ]:
# Check what is inside the zip
print(csv_data)
print(shape_data)

['nhgis0013_csv/nhgis0013_ds258_2020_block.csv', 'nhgis0013_csv/nhgis0013_ds258_2020_block_codebook.txt', 'nhgis0013_csv/nhgis0013_ds172_2010_block.csv', 'nhgis0013_csv/nhgis0013_ds172_2010_block_codebook.txt']
['nhgis0013_shape/nhgis0013_shapefile_tl2020_720_block_2020.zip']


In [4]:
# Read zip data file in the extract
with ZipFile(shapefile) as outer_zip: # Shapefile data has 2 zipped layers
    with outer_zip.open(shape_data[0]) as inner_zip:
        with ZipFile(inner_zip) as inner:
            inner.extractall(DOWNLOAD_DIR) #Extract the shapefile into the data folder
            

In [4]:
#Read csv data

with ZipFile(csv) as z:
    with z.open(csv_data[0]) as f: 
        df_2020 = pd.read_csv(f)
    with z.open(csv_data[2]) as f:
        df_2010 = pd.read_csv(f)

In [5]:
shp_df = pd.DataFrame.spatial.from_featureclass(location = "./data/PR_block_2020.shp", sr = 3857)

In [6]:
## Using earthaccess to search and find 

url = "https://cmr.earthdata.nasa.gov/search/collections?concept_id[]=C123456-LPDAAC_ECS"

query = ea.search_datasets(keyword = "LECZ")
for collection in query[:10]:
    pprint.pprint(collection.summary(), sort_dicts=True, indent=4)
    print('')  # Add a space between collections for readability

{   'concept-id': 'C1000000400-SEDAC',
    'file-type': "[{'Format': 'CSV', 'Fees': '0'}, {'Format': 'Excel', 'Fees': "
                 "'0'}, {'Format': 'PDF', 'Fees': '0'}, {'Format': 'PNG', "
                 "'Fees': '0'}, {'Format': 'WMS', 'Fees': '0'}]",
    'get-data': [   'https://search.earthdata.nasa.gov/search/granules?p=C1000000400-SEDAC'],
    'short-name': 'CIESIN_SEDAC_LECZ_URPLAEV2',
    'version': '2.00'}

{   'concept-id': 'C3104974973-SEDAC',
    'file-type': "[{'Format': 'Excel', 'Fees': '0'}, {'Format': 'PDF', 'Fees': "
                 "'0'}, {'Format': 'PNG', 'Fees': '0'}]",
    'get-data': [   'https://search.earthdata.nasa.gov/search/granules?p=C3104974973-SEDAC'],
    'short-name': 'CIESIN_SEDAC_LECZ_DELTA_V1',
    'version': '1.00'}

{   'concept-id': 'C2187533258-SEDAC',
    'file-type': "[{'Format': 'Excel', 'Fees': '0'}, {'Format': 'GeoTIFF', "
                 "'Fees': '0'}, {'Format': 'PDF', 'Fees': '0'}, {'Format': "
                 "'PNG', 'Fees': '0

In [ ]:
# #Login to the Earthdata
# # Create a username and password in .env file where you stored IPUMS API key
# auth = ea.login(strategy= "environment")

# #Download the data through the Earthdata
# Query = (
#     ea.granule_query()
#     .short_name("CIESIN_SEDAC_LECZ_URPLAEV3")
#     .debug(True)
# )

# print(f"Granule hits: {Query.hits()}")

# # Get the first 10 granules
# granules = Query.get(10)

# #Check if the granules are cloud hosted
# granules[1].cloud_hosted
# download_file = ea.download(granules[1], local_path="./data")

Granule hits: 2


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
#Load data to the notebook

tiff = [f for f in file_list if f.endswith('-geotiff.zip')]

tiff_path = os.path.join(f"{DOWNLOAD_DIR}/{tiff[0]}")


with ZipFile(tiff_path) as z:
    z.extractall(DOWNLOAD_DIR)

In [21]:
file_list

['nhgis0013_csv.zip',
 'nhgis0013_shape.zip',
 'PR_block_2020.cpg',
 'PR_block_2020.dbf',
 'PR_block_2020.GISJOIN.atx',
 'PR_block_2020.prj',
 'PR_block_2020.sbn',
 'PR_block_2020.sbx',
 'PR_block_2020.shp',
 'PR_block_2020.shp.xml',
 'PR_block_2020.shx']

In [6]:
df_2010.shape, df_2020.shape, shp_df.shape

((77189, 62), (41987, 63), (41320, 21))

In [ ]:
df_2020.head()

,GISJOIN,YEAR,STUSAB,GEOID,GEOCODE,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,...,NAME,FUNCSTAT,INTPTLAT,INTPTLON,LSADC,UGA,U9V001,U9X001,U9X002,U9X003
0,G72000109563001000,2020,PR,1000000US720019563001000,720019563001000,9,0,Puerto Rico,72,Adjuntas Municipio,...,Block 1000,S,18.231874,-66.771935,BK,99999,68,68,60,8
1,G72000109563001001,2020,PR,1000000US720019563001001,720019563001001,9,0,Puerto Rico,72,Adjuntas Municipio,...,Block 1001,S,18.220806,-66.776888,BK,99999,46,46,45,1
2,G72000109563001002,2020,PR,1000000US720019563001002,720019563001002,9,0,Puerto Rico,72,Adjuntas Municipio,...,Block 1002,S,18.221548,-66.782006,BK,99999,22,22,11,11
3,G72000109563001003,2020,PR,1000000US720019563001003,720019563001003,9,0,Puerto Rico,72,Adjuntas Municipio,...,Block 1003,S,18.218566,-66.764885,BK,99999,43,43,32,11
4,G72000109563001004,2020,PR,1000000US720019563001004,720019563001004,9,0,Puerto Rico,72,Adjuntas Municipio,...,Block 1004,S,18.213141,-66.761705,BK,99999,31,31,31,0


In [7]:
#Rename the columns the human readable names
# The NHGIS codes are as follows in the documentation which is downloaded from the IPUMS API 

'''    Table 1:     Housing Units
    Universe:    Housing units
    Source code: H1
    NHGIS code:  U9V
        U9V001:      Total
 
    Table 2:     Occupancy Status
    Universe:    Housing units
    Source code: H3
    NHGIS code:  U9X
        U9X001:      Total
        U9X002:      Occupied
        U9X003:      Vacant
'''

# Get the layer from the published data

gis = GIS()
layer = gis.content.get("1322372408e744b7a384cda121be8814").layers[0] # The layer of PR block 2020
sedf = pd.DataFrame.spatial.from_layer(layer) # Convert the layer to a spatial dataframe to use it in the analysis




rename = {
    "U9V001": "Total_Housing",
    "U9X001": "Total_Housing_Units",
    "U9X002": "Occupied",
    "U9X003": "Vacant"
}

df_2010.rename(columns = rename, inplace = True)
df_2020.rename(columns = rename, inplace = True)


pr_sedf = sedf.merge(df_2020[["GISJOIN", "Total_Housing", "Total_Housing_Units", "Occupied", "Vacant"]], on = "GISJOIN", how = "inner")
pr_sedf.head()

,FID,GISJOIN,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,...,INTPTLON20,Shape_Leng,Shape_Area,Shape__Area,Shape__Length,SHAPE,Total_Housing,Total_Housing_Units,Occupied,Vacant
0,1,G72000109563001000,72,001,956300,1000,720019563001000,Block 1000,G5040,,...,-066.7719354,15320.205987,5732654.69615,6389121.943359,16135.340919,"{""rings"": [[[-7433468.03304438, 2066025.482938...",68,68,60,8
1,2,G72000109563001001,72,001,956300,1001,720019563001001,Block 1001,G5040,,...,-066.7768876,2695.184677,203981.899808,227312.088867,2809.128561,"{""rings"": [[[-7432871.58310674, 2063230.193913...",46,46,45,1
2,3,G72000504007002004,72,005,400700,2004,720054007002004,Block 2004,G5040,,...,-067.1558524,910.060507,23634.820744,26406.524414,974.914527,"{""rings"": [[[-7475651.00912771, 2090141.650471...",13,13,13,0
3,4,G72000109563001002,72,001,956300,1002,720019563001002,Block 1002,G5040,,...,-066.7820065,3462.974754,510326.461548,568699.155273,3677.834548,"{""rings"": [[[-7433820.24808916, 2063740.115163...",22,22,11,11
4,5,G72000109563001003,72,001,956300,1003,720019563001003,Block 1003,G5040,,...,-066.7648852,2372.120012,144307.637692,160808.522461,2472.028213,"{""rings"": [[[-7431799.01940358, 2062908.965386...",43,43,32,11


In [8]:
lecz_layer = gis.content.search("Low Elevation Coastal Zones derived from MERIT-DEM", item_type="Imagery Layer")
for item in lecz_layer:
    display(item)
lecz = lecz_layer[0]

<Item title:"Low Elevation Coastal Zones derived from MERIT-DEM" type:Imagery Layer owner:NASA_Earthdata>

<Item title:"Global population living within 10m of sea level (2015)" type:Tiled Imagery Layer owner:dasbury_storymaps>

In [58]:
m1 = Map("Puerto Rico")
m1.content.add(lecz)
m1.content.add(pr_sedf, options= {"opacity": 0.75})


ValueError: 'layer' must be one of these types: CatalogFootprintLayer, CSVLayer, FeatureLayer, GeoJSONLayer, KnowledgeGraphSublayer, OGCFeatureLayer, OrientedImageryLayer, SceneLayer, StreamLayer, WFSLayer

In [60]:
renderer_manager = m1.content.renderer(0)
smart_mapping_manager = renderer_manager.smart_mapping()
smart_mapping_manager.class_breaks_renderer(
    break_type="color",
    field="Vacant",
)

In [9]:
from arcgis.map.renderers import (
    ClassBreaksRenderer,
    ClassBreakInfo,
    UniqueValueRenderer,
    UniqueValueInfo,
    SizeInfoVisualVariable,
)

from arcgis.map.symbols import SimpleLineSymbolEsriSLS, SimpleFillSymbolEsriSFS

class_break_infos = [
    ClassBreakInfo(
        class_max_value=5,
        label= "< 5m",
        symbol= SimpleFillSymbolEsriSFS(
            **{
                "type":"esriSFS",
                "style": "esriSFSSolid",
                "color": [0,77, 230],
                "outline": {"color" : [153, 153, 153, 255], "width": 0.5},
            }
        ),
    ),
    ClassBreakInfo(
        class_max_value=10,
        label= "< 10m",
        symbol= SimpleFillSymbolEsriSFS(
            **{
                "type":"esriSFS",
                "style": "esriSFSSolid",
                "color": [77, 166, 255],
                "outline": {"color" : [153, 153, 153, 255], "width": 0.5},
            }
        ),
    ),
]

class_break_renderer = ClassBreaksRenderer(
    field="Vacant",
    class_break_infos=class_break_infos,
    normalizationType= None,
)

In [13]:
m2 = Map("Puerto Rico")
m2.content.add(pr_sedf, drawing_info={"renderer":class_break_renderer.dict()})

In [14]:
m2

Map(center=[2063620.7690671808, -7399290.049500733], extent={'xmin': -7498920.993760712, 'ymin': 1958993.94058…

In [17]:
pr_sedf[["Total_Housing", "Vacant"]]

,Total_Housing,Vacant
0,68,8
1,46,1
2,13,0
3,22,11
4,43,11
...,...,...
41315,6,6
41316,4,3
41317,27,0
41318,111,22
